# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Mod 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

# Restating Goal: 

Upon completion of this lab, *each unique team* in this dataset should have a record in the MongoDB instance containing the following information:
<ol>
    
  * The name of the team
  * The total number of goals scored by the team during the 2011 season
  * The total number of wins the team earned during the 2011 season 
  * A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
  * The team's win percentage on days where it was raining during games in the 2011 season.  </ol>

## Workplan  :

### * Query the SQL database  https://www.kaggle.com/laudanum/footballdelphi
    Get: 
        * Team Name
        * Goals Scored (2011)
        * Number of Wins (2011)
        * Dates (for use for weather)
        
### * Calculate summary statistics
    Calc:
        * Total Goals Scored (2011) (sum)
        * Total Number of Wins (2011) (sum)
    Group By
        * Team Name
### * Get the weather data from the DarkSky API  https://darksky.net/dev  
    Class WeatherGetter to obtain weather for Berlin, Germany containing 'rain' corresponding to dates when games were played.
### * Load the data into MongoDB
    Class: MongoHandler to: load team into MongoDB
        




## Import Libraries

In [48]:
import sqlite3
import pandas as pd
import bs4 as BeautifulSoup
import numpy as np
import requests
import pymongo
import time
import matplotlib as plt
import json

## OBTAIN First dataset:

In [49]:
conn = sqlite3.connect('''database.sqlite''')
c = conn.cursor()

In [50]:
c.execute('''select * from matches where season = 2011''')

matches = pd.DataFrame(c.fetchall())
matches.columns = [i[0] for i in c.description]
matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [61]:
gamedates = [x for x in matches['Date'].unique()]
len(gamedates)


165

In [82]:
type(gamedates)

list

In [63]:
display(matches)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A


In [64]:
matches['Match_ID'].nunique()

992

In [65]:
len(matches)

992

In [ ]:
## just realized that A means away team won and H means home team won....

In [66]:
### this will build a dictionary for each team
## by taking all the uniuque home teams since every team plays at least once at home.
all_teams = matches['HomeTeam'].unique()
print("# of Unique Teams: {}".format(len(all_teams)))
teams_data = {}
for team in all_teams:
    data = {'total_matches_2011': 0, 
           'total_wins_2011': 0,
           'total_losses_2011': 0,
           'rain_wins_2011': 0,
           'rain_losses_2011': 0,
           'total_goals_2011': 0}
    teams_data[team] = data
print(len(teams_data))

# of Unique Teams: 56
56


In [ ]:
def TeamStats(df):
    self.hometeam
    self.awayteam
    
    Get: 
    * Team Name
    * Goals Scored (2011)
    * Number of Wins (2011)
    * Dates (for use for weather)
    

### OBTAIN Second Dataset

In [37]:
#https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/aosika/.secret/ds_api.json")

api_key = keys['api_key']

In [40]:
response = requests.get(f"https://api.darksky.net/forecast/{api_key}/52.5200,13.4050,20120331?exclude=currently,minutely,hourly,alerts,flags")


In [38]:
print(api_key)

49a70c8208df0fda1840a3ebb00ff142


In [39]:
type(api_key)

str

In [41]:
response.json()

{'latitude': 52.52,
 'longitude': 13.405,
 'timezone': 'Europe/Berlin',
 'daily': {'data': [{'time': 20041200,
    'sunriseTime': 20059200,
    'sunsetTime': 20110920,
    'moonPhase': 0.68,
    'windSpeed': 9.12,
    'windBearing': 108,
    'cloudCover': 0.84,
    'uvIndex': 4,
    'uvIndexTime': 20084100}]},
 'offset': 1}

In [79]:
test_date = matches['Date'][0]
test_date

'2012-03-31'

In [91]:
import requests
def api_request(time,verbose=1):
    # request_url = "https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]"
    lat =52.5200
    long= 13.4050

    time_suffix = "T12:00:00"
    check_date = time+time_suffix

    template_url= f"https://api.darksky.net/forecast/{api_key}/{lat},{long},{check_date}?exclude=currently,minutely,hourly,alerts,flags"
    if verbose==2:
        print(template_url)
    
    response = requests.get(template_url )
    
    if response.status_code == 200:
        if verbose>0: 
            print('Success')
    else: 
        print(f"Error: status code = {response.status_code}")
    
    return response

In [92]:
api_request(test_date, verbose=2)

https://api.darksky.net/forecast/49a70c8208df0fda1840a3ebb00ff142/52.52,13.405,2012-03-31T12:00:00?exclude=currently,minutely,hourly,alerts,flags
Success


<Response [200]>

In [93]:
resp = api_request(test_date,verbose=2)

https://api.darksky.net/forecast/49a70c8208df0fda1840a3ebb00ff142/52.52,13.405,2012-03-31T12:00:00?exclude=currently,minutely,hourly,alerts,flags
Success


In [94]:
r_dict = resp.json()
r_dict.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'daily', 'offset'])

In [97]:
# fs.ds.dict_dropdown(r_dict)
r_dict['daily']['data'][0]['icon']

'rain'

In [121]:

# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    
    def __init__(self,api_key, lat =52.5200, long= 13.4050,):
        """Sets the lat/long for the location, and stores api_key."""
        self.lat= lat
        self.long=long
        self.base_url  = "https://api.darksky.net/forecast/"
        self.api_key = api_key
        
    
    def get_weather_for_date(self, date,verbose=1):
        """Checks dark_sky api for weather from date. """
        import requests
        # request_url = "https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]"

        time_suffix = "T12:00:00"
        check_date = date+time_suffix

        template_url= f"https://api.darksky.net/forecast/{self.api_key}/{self.lat},{self.long},{check_date}"
        if verbose==2:
            print(template_url)

        ## Get and check response
        response = requests.get(template_url )
        if response.status_code == 200:
            
            if verbose>0: 
                print('Success')
        else: 
            print(f"Error: status code = {response.status_code}")
            
        # call helper method
        return self._get_weather_from_response(response)
    
    def _get_weather_from_response(self,response):
        """Extracts pricipProbability from dark_sky api response"""
        r_dict = resp.json()
        return r_dict['daily']['data'][0]['precipProbability']
        
    def get_weather_for_date_list(self,dates):    
        import time
        weather_list = []
        
        for date in dates:
            time.sleep(0.5)
            weather_list.append(self.get_weather_for_date(date))
            
            
#         self._weather_list     
        return weather_list

In [122]:
wget = WeatherGetter(api_key)
weather_results = wget.get_weather_for_date_list(gamedates)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [123]:
len(weather_results)

165

In [124]:
weather_dict = dict(zip(gamedates,weather_results))
weather_dict

{'2012-03-31': 0.92,
 '2011-12-11': 0.92,
 '2011-08-13': 0.92,
 '2011-11-27': 0.92,
 '2012-02-18': 0.92,
 '2012-01-20': 0.92,
 '2012-02-04': 0.92,
 '2012-04-21': 0.92,
 '2011-09-18': 0.92,
 '2011-10-23': 0.92,
 '2011-10-01': 0.92,
 '2012-03-03': 0.92,
 '2011-08-27': 0.92,
 '2012-03-17': 0.92,
 '2011-11-06': 0.92,
 '2012-05-05': 0.92,
 '2012-04-11': 0.92,
 '2011-12-17': 0.92,
 '2012-02-03': 0.92,
 '2011-10-29': 0.92,
 '2012-01-22': 0.92,
 '2011-12-03': 0.92,
 '2012-04-14': 0.92,
 '2012-03-25': 0.92,
 '2012-03-10': 0.92,
 '2012-04-07': 0.92,
 '2011-11-19': 0.92,
 '2011-10-14': 0.92,
 '2011-09-24': 0.92,
 '2012-04-28': 0.92,
 '2011-12-18': 0.92,
 '2012-03-02': 0.92,
 '2012-03-16': 0.92,
 '2012-02-17': 0.92,
 '2011-08-06': 0.92,
 '2011-11-04': 0.92,
 '2011-09-16': 0.92,
 '2011-07-15': 0.92,
 '2012-05-06': 0.92,
 '2012-02-11': 0.92,
 '2011-10-02': 0.92,
 '2012-03-30': 0.92,
 '2011-10-22': 0.92,
 '2011-08-26': 0.92,
 '2011-08-07': 0.92,
 '2012-02-24': 0.92,
 '2011-07-17': 0.92,
 '2012-02-12'

In [ ]:
class MongoHandler(team):
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!